In [1]:
from pytorch_pretrained_bert.tokenization import BertTokenizer, WordpieceTokenizer
from pytorch_pretrained_bert.modeling import BertForPreTraining, BertPreTrainedModel, BertModel, BertConfig, BertForMaskedLM, BertForSequenceClassification
from pathlib import Path
import torch
import re
from torch import Tensor
from torch.nn import BCEWithLogitsLoss
from fastai.text import Tokenizer, Vocab
import pandas as pd
import collections
import os
import pdb
from tqdm import tqdm, trange
import sys
import random
import numpy as np
#import apex
from sklearn.model_selection import train_test_split
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from sklearn.metrics import roc_curve, auc


from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from pytorch_pretrained_bert.optimization import BertAdam

In [2]:
import h5py,json

SETTING UP LOG

Specify location and version to run the code on log_filename

In [3]:
import socket 

hostname = socket. gethostname()
version_id=1

log_filename = 'bert-cpc-multilabel-classification{0}_at_{1}_.full_nodes.log'.format(version_id,hostname)

import logging

logger = logging.getLogger(__name__)

logger.setLevel(logging.INFO)

# create file handler which logs even debug messages
fh = logging.FileHandler(log_filename)

# create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)
# add the handlers to the logger

logger.addHandler(fh)

logger.info('Started')



In [4]:
class PatentTransformer(object):
    def __init__(self, label_list, max_seq_length, tokenizer):
        self.label_map = {label : i for i, label in enumerate(label_list)}
        logger.info('label_map={0}'.format(self.label_map))
        
        self.max_seq_length = max_seq_length
        logger.info('max_seq_length={0}'.format(self.max_seq_length))
        
        
        self.tokenizer = tokenizer
    
    def transform(self,patent_case):
        '''
            transform a json patent_case to a feature for BERT, a tuple of Tensor
        '''
        example = self.__get_example__(patent_case)
        #logger.info('example={0}'.format(example.guid))
        
        feature = self.__get_feature__(example)
        #logger.info('feature={0}'.format(feature.label_ids))
        
        return feature
    
    def __get_example__(self, patent_case):
        '''
            given a patent_case as a json, the method return a features for BERT
        '''
        #join a list of text with new line as a seperator
        fulltext_list = [patent_case['titleText'], patent_case['abstractText']] + patent_case['claimText']
        
        guid = patent_case['pubId']
        text_a = '\n'.join(fulltext_list)
        labels = patent_case['cpcList']
        return InputExample(guid=guid, text_a=text_a, labels=labels)
        
    def __get_feature__(self, example):
        """
            convert a single example (InputExample to a single feature instance required by BERT)

        """
    
        tokens_a = self.tokenizer.tokenize(example.text_a)

        tokens_b = None
        if example.text_b:
            tokens_b = self.tokenizer.tokenize(example.text_b)
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            _truncate_seq_pair(tokens_a, tokens_b, self.max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > self.max_seq_length - 2:
                tokens_a = tokens_a[:(self.max_seq_length - 2)]

        # The convention in BERT is:
        # (a) For sequence pairs:
        #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
        #  type_ids: 0   0  0    0    0     0       0 0    1  1  1  1   1 1
        # (b) For single sequences:
        #  tokens:   [CLS] the dog is hairy . [SEP]
        #  type_ids: 0   0   0   0  0     0 0
        #
        # Where "type_ids" are used to indicate whether this is the first
        # sequence or the second sequence. The embedding vectors for `type=0` and
        # `type=1` were learned during pre-training and are added to the wordpiece
        # embedding vector (and position vector). This is not *strictly* necessary
        # since the [SEP] token unambigiously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.
        #
        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens += tokens_b + ["[SEP]"]
            segment_ids += [1] * (len(tokens_b) + 1)

        input_ids = self.tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding = [0] * (self.max_seq_length - len(input_ids))
        input_ids += padding
        input_mask += padding
        segment_ids += padding

        assert len(input_ids) == self.max_seq_length
        assert len(input_mask) == self.max_seq_length
        assert len(segment_ids) == self.max_seq_length

        label_ids = [0.0]*len(self.label_map)
        for label in example.labels:
            label_ids[self.label_map[label]]=float(1)



        feature = InputFeatures(input_ids=input_ids,
                            input_mask=input_mask,
                            segment_ids=segment_ids,
                            label_ids=label_ids)
    
        return (torch.tensor(feature.input_ids,dtype=torch.long),\
                    torch.tensor(feature.input_mask,dtype=torch.long),\
                    torch.tensor(feature.segment_ids,dtype=torch.long), \
                    torch.tensor(feature.label_ids, dtype=torch.float))

class PatentDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe,transformer=None, size=-1):
        self.dataframe = dataframe
        self.transformer = transformer
        self.size = size
        
    def __getitem__(self, index):
        row = self.dataframe.loc[index,:] #a row can be used a a dictionary, just like a json object
        return self.transformer.transform(row)
    
    def __len__(self):
        if self.size == -1:
            return len(self.dataframe)
        
        return min(self.size,len(self.dataframe))
        

    
    

    
    



class HDF5Dataset(torch.utils.data.Dataset):
    def __init__(self, hdf5_dataset_file, hdf5_dataset_name, transformer=None, mode='r', size=-1):
        
        '''
            transformer: is a class that has transform(value) function that convert the value to a feature for BERT
        '''
        
        self.f = h5py.File(hdf5_dataset_file, mode)
        self.dataset = self.f[hdf5_dataset_name]
        self.transformer = transformer
        logger.info('dataset.name={0}'.format(self.dataset.name))
        
        self.size = size
        

    def __getitem__(self, index):
        
        value = self.dataset[index]
        logger.info('index={0}, value={1}'.format(index, value[:10]))
        
        #value is a json text
        try:
            cases = json.loads(value)
            feature = self.transformer.transform(cases)
            
            
            return feature
        except:
            logger.error('Unable to process index={0} with value={1}'.format(index, value))
            logger.info('Looking for replacement for {0}'.format(index))
            
            #Process to see to get an replacement for the value at index by looking at neighboards
            inc_index = index
            feature = None

            while (feature is None and inc_index < self.__len__()):
                feature = self.__getitem__(inc_index+1)
                if feature is not None:
                    logger.info('Found a replacement at {0}'.format(inc_index+1))

                    return feature
                inc_index +=1
                
            dec_index = index
            while (feature is None and dec_index >0):
                feature = self.__getitem__(dec_index-1)
                if feature is not None:
                    logger.info('Found a replacement at {0}'.format(dec_index-1))

                    return feature
                dec_index -=1
                
            logger.error("Unable to process due to empty value return, please clean up the dataset")
            
            
            
                

    def __len__(self):
        if self.size == -1:
            return self.dataset.size
        else:
            return self.size
    
    def __del__(self):
        if self.f:
            self.f.close()
    @staticmethod        
    def get_labels(filename = None):
        if filename is not None:
            return list(pd.read_csv(filename,header=None)[0].values)
        return None


In [5]:
DATA_PATH=Path('/home/bimhud/bert-cpc')
DATA_PATH.mkdir(exist_ok=True)

PATH=Path('/home/bimhud/bert-cpc')
PATH.mkdir(exist_ok=True)

model_state_dict = None

BERT_PRETRAINED_PATH = Path('/home/bimhud/python-workspace/bert/bert-models/uncased_L-24_H-1024_A-16/')

# BERT_FINETUNED_WEIGHTS = Path('../trained_model/toxic_comments')

PYTORCH_PRETRAINED_BERT_CACHE = BERT_PRETRAINED_PATH/'cache/'
PYTORCH_PRETRAINED_BERT_CACHE.mkdir(exist_ok=True)

# output_model_file = os.path.join(BERT_FINETUNED_WEIGHTS, "pytorch_model.bin")

# Load a trained model that you have fine-tuned
# model_state_dict = torch.load(output_model_file)

## Model Parameters

In [56]:
args = {
    "train_size": 10000,
    "eval_size": 100,
    "hdf5_dataset_file":'/data/cf_dataset/CPC/US_CPC_2010_2019_subgroup_200/bert-cpc/uspto_cpc.hdf5',
    'hdf5_train_dataset_name':'CPC_5_head_sample_per_subgroup',
    'hdf5_eval_dataset_name':'CPC_5_tail_sample_per_subgroup',
    'hdf5_test_dataset_name':'CPC_5_tail_sample_per_subgroup',
    "full_data_dir": DATA_PATH,
    "data_dir": DATA_PATH,
    "no_cuda": False,
    "bert_model": BERT_PRETRAINED_PATH,
    "output_dir": DATA_PATH,
    "max_seq_length": 512,
    "do_train": True,
    "do_eval": True,
    "do_lower_case": True,
    "train_batch_size": 64,
    "eval_batch_size": 64,
    "learning_rate": 3e-5,
    "num_train_epochs": 4.0,
    "warmup_proportion": 0.1,
    "no_cuda": False,
    "local_rank": -1,
    "seed": 42,
    "gradient_accumulation_steps": 1,
    "optimize_on_cpu": False,
    "fp16": False,
    "loss_scale": 128
}


###  Model Class

In [7]:
class BertForMultiLabelSequenceClassification(BertPreTrainedModel):
    """BERT model for classification.
    This module is composed of the BERT model with a linear layer on top of
    the pooled output.
    Params:
        `config`: a BertConfig class instance with the configuration to build a new model.
        `num_labels`: the number of classes for the classifier. Default = 2.
    Inputs:
        `input_ids`: a torch.LongTensor of shape [batch_size, sequence_length]
            with the word token indices in the vocabulary(see the tokens preprocessing logic in the scripts
            `extract_features.py`, `run_classifier.py` and `run_squad.py`)
        `token_type_ids`: an optional torch.LongTensor of shape [batch_size, sequence_length] with the token
            types indices selected in [0, 1]. Type 0 corresponds to a `sentence A` and type 1 corresponds to
            a `sentence B` token (see BERT paper for more details).
        `attention_mask`: an optional torch.LongTensor of shape [batch_size, sequence_length] with indices
            selected in [0, 1]. It's a mask to be used if the input sequence length is smaller than the max
            input sequence length in the current batch. It's the mask that we typically use for attention when
            a batch has varying length sentences.
        `labels`: labels for the classification output: torch.LongTensor of shape [batch_size]
            with indices selected in [0, ..., num_labels].
    Outputs:
        if `labels` is not `None`:
            Outputs the CrossEntropy classification loss of the output with the labels.
        if `labels` is `None`:
            Outputs the classification logits of shape [batch_size, num_labels].
    Example usage:
    ```python
    # Already been converted into WordPiece token ids
    input_ids = torch.LongTensor([[31, 51, 99], [15, 5, 0]])
    input_mask = torch.LongTensor([[1, 1, 1], [1, 1, 0]])
    token_type_ids = torch.LongTensor([[0, 0, 1], [0, 1, 0]])
    config = BertConfig(vocab_size_or_config_json_file=32000, hidden_size=768,
        num_hidden_layers=12, num_attention_heads=12, intermediate_size=3072)
    num_labels = 2
    model = BertForSequenceClassification(config, num_labels)
    logits = model(input_ids, token_type_ids, input_mask)
    ```
    """
    def __init__(self, config, num_labels=2):
        super(BertForMultiLabelSequenceClassification, self).__init__(config)
        self.num_labels = num_labels
        self.bert = BertModel(config)
        self.dropout = torch.nn.Dropout(config.hidden_dropout_prob)
        self.classifier = torch.nn.Linear(config.hidden_size, num_labels)
        self.apply(self.init_bert_weights)

    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):
        _, pooled_output = self.bert(input_ids, token_type_ids, attention_mask, output_all_encoded_layers=False)
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        if labels is not None:
            loss_fct = BCEWithLogitsLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1, self.num_labels))
            return loss
        else:
            return logits
        
    def freeze_bert_encoder(self):
        for param in self.bert.parameters():
            param.requires_grad = False
    
    def unfreeze_bert_encoder(self):
        for param in self.bert.parameters():
            param.requires_grad = True

## Data representation class

In [8]:
class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, labels=None):
        """Constructs a InputExample.

        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
            labels: (Optional) [string]. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.labels = labels


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_ids):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_ids = label_ids

In [9]:
def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

## Metric functions

In [10]:
def accuracy(out, labels):
    outputs = np.argmax(out, axis=1)
    return np.sum(outputs == labels)

def accuracy_thresh(y_pred:Tensor, y_true:Tensor, thresh:float=0.5, sigmoid:bool=True):
    "Compute accuracy when `y_pred` and `y_true` are the same size."
    if sigmoid: y_pred = y_pred.sigmoid()
#     return ((y_pred>thresh)==y_true.byte()).float().mean().item()
    return np.mean(((y_pred>thresh)==y_true.byte()).float().cpu().numpy(), axis=1).sum()


def fbeta(y_pred:Tensor, y_true:Tensor, thresh:float=0.2, beta:float=2, eps:float=1e-9, sigmoid:bool=True):
    "Computes the f_beta between `preds` and `targets`"
    beta2 = beta ** 2
    if sigmoid: y_pred = y_pred.sigmoid()
    y_pred = (y_pred>thresh).float()
    y_true = y_true.float()
    TP = (y_pred*y_true).sum(dim=1)
    prec = TP/(y_pred.sum(dim=1)+eps)
    rec = TP/(y_true.sum(dim=1)+eps)
    res = (prec*rec)/(prec*beta2+rec+eps)*(1+beta2)
    return res.mean().item()

## Training warmup 

In [11]:
def warmup_linear(x, warmup=0.002):
    if x < warmup:
        return x/warmup
    return 1.0 - x

In [12]:


# Setup GPU parameters

if args["local_rank"] == -1 or args["no_cuda"]:
    device = torch.device("cuda" if torch.cuda.is_available() and not args["no_cuda"] else "cpu")
    n_gpu = torch.cuda.device_count()
#     n_gpu = 1
else:
    torch.cuda.set_device(args['local_rank'])
    device = torch.device("cuda", args['local_rank'])
    n_gpu = 1
    # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
    torch.distributed.init_process_group(backend='nccl')
logger.info("device: {} n_gpu: {}, distributed training: {}, 16-bits training: {}".format(
        device, n_gpu, bool(args['local_rank'] != -1), args['fp16']))

In [13]:
torch.set_num_threads(30)
torch.get_num_threads()

30

In [14]:
args['train_batch_size'] = int(args['train_batch_size'] / args['gradient_accumulation_steps'])

In [15]:
random.seed(args['seed'])
np.random.seed(args['seed'])
torch.manual_seed(args['seed'])
if n_gpu > 0:
    torch.cuda.manual_seed_all(args['seed'])

# Loading labels

In [16]:
label_list = HDF5Dataset.get_labels(filename=os.path.join(args['data_dir'],'classes.txt'))
num_labels = len(label_list)
num_labels

143499

### Load training data

In [17]:
tokenizer = BertTokenizer.from_pretrained(args['bert_model'], do_lower_case=args['do_lower_case'])

In [18]:
#This will support big dataset


transformer = PatentTransformer(label_list, args['max_seq_length'], tokenizer)

train_data = HDF5Dataset(args['hdf5_dataset_file'], args['hdf5_train_dataset_name'], transformer=transformer, size=args['train_size'])
train_dataloader = DataLoader(train_data, shuffle=True, batch_size=args['train_batch_size'],num_workers=0)

In [ ]:
logger.info('len(train_data)={0}'.format(len(train_data)))

In [ ]:
# Prepare model
def get_model():
#     pdb.set_trace()
    if model_state_dict:
        model = BertForMultiLabelSequenceClassification.from_pretrained(args['bert_model'], num_labels = num_labels, \
                                                                        state_dict=model_state_dict)
    else:
        model = BertForMultiLabelSequenceClassification.from_pretrained(args['bert_model'], num_labels = num_labels)
    return model

model = get_model()

if args['fp16']:
    model.half()
model.to(device)
if args['local_rank'] != -1:
    try:
        from apex.parallel import DistributedDataParallel as DDP
    except ImportError:
        raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use distributed and fp16 training.")

    model = DDP(model)
elif n_gpu > 1:
    model = torch.nn.DataParallel(model)

In [ ]:
logger.info('model={0}'.format(model))

In [ ]:
from torch.optim.lr_scheduler import _LRScheduler, Optimizer

class CyclicLR(object):
    """Sets the learning rate of each parameter group according to
    cyclical learning rate policy (CLR). The policy cycles the learning
    rate between two boundaries with a constant frequency, as detailed in
    the paper `Cyclical Learning Rates for Training Neural Networks`_.
    The distance between the two boundaries can be scaled on a per-iteration
    or per-cycle basis.
    Cyclical learning rate policy changes the learning rate after every batch.
    `batch_step` should be called after a batch has been used for training.
    To resume training, save `last_batch_iteration` and use it to instantiate `CycleLR`.
    This class has three built-in policies, as put forth in the paper:
    "triangular":
        A basic triangular cycle w/ no amplitude scaling.
    "triangular2":
        A basic triangular cycle that scales initial amplitude by half each cycle.
    "exp_range":
        A cycle that scales initial amplitude by gamma**(cycle iterations) at each
        cycle iteration.
    This implementation was adapted from the github repo: `bckenstler/CLR`_
    Args:
        optimizer (Optimizer): Wrapped optimizer.
        base_lr (float or list): Initial learning rate which is the
            lower boundary in the cycle for eachparam groups.
            Default: 0.001
        max_lr (float or list): Upper boundaries in the cycle for
            each parameter group. Functionally,
            it defines the cycle amplitude (max_lr - base_lr).
            The lr at any cycle is the sum of base_lr
            and some scaling of the amplitude; therefore
            max_lr may not actually be reached depending on
            scaling function. Default: 0.006
        step_size (int): Number of training iterations per
            half cycle. Authors suggest setting step_size
            2-8 x training iterations in epoch. Default: 2000
        mode (str): One of {triangular, triangular2, exp_range}.
            Values correspond to policies detailed above.
            If scale_fn is not None, this argument is ignored.
            Default: 'triangular'
        gamma (float): Constant in 'exp_range' scaling function:
            gamma**(cycle iterations)
            Default: 1.0
        scale_fn (function): Custom scaling policy defined by a single
            argument lambda function, where
            0 <= scale_fn(x) <= 1 for all x >= 0.
            mode paramater is ignored
            Default: None
        scale_mode (str): {'cycle', 'iterations'}.
            Defines whether scale_fn is evaluated on
            cycle number or cycle iterations (training
            iterations since start of cycle).
            Default: 'cycle'
        last_batch_iteration (int): The index of the last batch. Default: -1
    Example:
        >>> optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
        >>> scheduler = torch.optim.CyclicLR(optimizer)
        >>> data_loader = torch.utils.data.DataLoader(...)
        >>> for epoch in range(10):
        >>>     for batch in data_loader:
        >>>         scheduler.batch_step()
        >>>         train_batch(...)
    .. _Cyclical Learning Rates for Training Neural Networks: https://arxiv.org/abs/1506.01186
    .. _bckenstler/CLR: https://github.com/bckenstler/CLR
    """

    def __init__(self, optimizer, base_lr=1e-3, max_lr=6e-3,
                 step_size=2000, mode='triangular', gamma=1.,
                 scale_fn=None, scale_mode='cycle', last_batch_iteration=-1):

#         if not isinstance(optimizer, Optimizer):
#             raise TypeError('{} is not an Optimizer'.format(
#                 type(optimizer).__name__))
        self.optimizer = optimizer

        if isinstance(base_lr, list) or isinstance(base_lr, tuple):
            if len(base_lr) != len(optimizer.param_groups):
                raise ValueError("expected {} base_lr, got {}".format(
                    len(optimizer.param_groups), len(base_lr)))
            self.base_lrs = list(base_lr)
        else:
            self.base_lrs = [base_lr] * len(optimizer.param_groups)

        if isinstance(max_lr, list) or isinstance(max_lr, tuple):
            if len(max_lr) != len(optimizer.param_groups):
                raise ValueError("expected {} max_lr, got {}".format(
                    len(optimizer.param_groups), len(max_lr)))
            self.max_lrs = list(max_lr)
        else:
            self.max_lrs = [max_lr] * len(optimizer.param_groups)

        self.step_size = step_size

        if mode not in ['triangular', 'triangular2', 'exp_range'] \
                and scale_fn is None:
            raise ValueError('mode is invalid and scale_fn is None')

        self.mode = mode
        self.gamma = gamma

        if scale_fn is None:
            if self.mode == 'triangular':
                self.scale_fn = self._triangular_scale_fn
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = self._triangular2_scale_fn
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = self._exp_range_scale_fn
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode

        self.batch_step(last_batch_iteration + 1)
        self.last_batch_iteration = last_batch_iteration

    def batch_step(self, batch_iteration=None):
        if batch_iteration is None:
            batch_iteration = self.last_batch_iteration + 1
        self.last_batch_iteration = batch_iteration
        for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
            param_group['lr'] = lr

    def _triangular_scale_fn(self, x):
        return 1.

    def _triangular2_scale_fn(self, x):
        return 1 / (2. ** (x - 1))

    def _exp_range_scale_fn(self, x):
        return self.gamma**(x)

    def get_lr(self):
        step_size = float(self.step_size)
        cycle = np.floor(1 + self.last_batch_iteration / (2 * step_size))
        x = np.abs(self.last_batch_iteration / step_size - 2 * cycle + 1)

        lrs = []
        param_lrs = zip(self.optimizer.param_groups, self.base_lrs, self.max_lrs)
        for param_group, base_lr, max_lr in param_lrs:
            base_height = (max_lr - base_lr) * np.maximum(0, (1 - x))
            if self.scale_mode == 'cycle':
                lr = base_lr + base_height * self.scale_fn(cycle)
            else:
                lr = base_lr + base_height * self.scale_fn(self.last_batch_iteration)
            lrs.append(lr)
        return lrs

In [ ]:
# Prepare optimizer


num_train_steps = int(
    len(train_data) / args['train_batch_size'] / args['gradient_accumulation_steps'] * args['num_train_epochs'])

logger.info('num_train_steps={0}'.format(num_train_steps))

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
t_total = num_train_steps
if args['local_rank'] != -1:
    t_total = t_total // torch.distributed.get_world_size()
if args['fp16']:
    try:
        from apex.optimizers import FP16_Optimizer
        from apex.optimizers import FusedAdam
    except ImportError:
        raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use distributed and fp16 training.")

    optimizer = FusedAdam(optimizer_grouped_parameters,
                          lr=args['learning_rate'],
                          bias_correction=False,
                          max_grad_norm=1.0)
    if args['loss_scale'] == 0:
        optimizer = FP16_Optimizer(optimizer, dynamic_loss_scale=True)
    else:
        optimizer = FP16_Optimizer(optimizer, static_loss_scale=args['loss_scale'])

else:
    optimizer = BertAdam(optimizer_grouped_parameters,
                         lr=args['learning_rate'],
                         warmup=args['warmup_proportion'],
                         t_total=t_total)

scheduler = CyclicLR(optimizer, base_lr=2e-5, max_lr=5e-5, step_size=2500, last_batch_iteration=0)

In [106]:
#import pdb

def eval():
    args['output_dir'].mkdir(exist_ok=True)
    
    #This will support big dataset
    transformer = PatentTransformer(label_list, args['max_seq_length'], tokenizer)

    eval_data = HDF5Dataset(args['hdf5_dataset_file'], args['hdf5_eval_dataset_name'], transformer=transformer, \
                            size=args['eval_size'])
    
    eval_dataloader = DataLoader(eval_data, shuffle=True, batch_size=args['eval_batch_size'], num_workers=0)
  
    
    all_logits = None
    all_labels = None
    
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    count=0
    for input_ids, input_mask, segment_ids, label_ids in tqdm(eval_dataloader,total=args['eval_size']):
        
        #pdb.set_trace()
        
        logger.info('count={0}'.format(count))
        count+=1
        input_ids = input_ids.to(device)
        input_mask = input_mask.to(device)
        segment_ids = segment_ids.to(device)
        label_ids = label_ids.to(device)

        with torch.no_grad():
            tmp_eval_loss = model(input_ids, segment_ids, input_mask, label_ids)
            logits = model(input_ids, segment_ids, input_mask)

#         logits = logits.detach().cpu().numpy()
#         label_ids = label_ids.to('cpu').numpy()
#         tmp_eval_accuracy = accuracy(logits, label_ids)
        tmp_eval_accuracy = accuracy_thresh(logits, label_ids)
        if all_logits is None:
            all_logits = logits.detach().cpu().numpy()
        else:
            all_logits = np.concatenate((all_logits, logits.detach().cpu().numpy()), axis=0)
            
        if all_labels is None:
            all_labels = label_ids.detach().cpu().numpy()
        else:    
            all_labels = np.concatenate((all_labels, label_ids.detach().cpu().numpy()), axis=0)
        

        eval_loss += tmp_eval_loss.mean().item()
        eval_accuracy += tmp_eval_accuracy

        nb_eval_examples += input_ids.size(0)
        nb_eval_steps += 1

    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_examples
    
    
    
    
#     ROC-AUC calcualation
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    
    for i in range(num_labels):
        fpr[i], tpr[i], _ = roc_curve(all_labels[:, i], all_logits[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
        
    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(all_labels.ravel(), all_logits.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

    result = {'eval_loss': eval_loss,
              'eval_accuracy': eval_accuracy,
#               'loss': tr_loss/nb_tr_steps,
              'roc_auc': roc_auc  }

    output_eval_file = os.path.join(args['output_dir'], "eval_results.txt")
    with open(output_eval_file, "w") as writer:
        logger.info("***** Eval results *****")
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))
#             writer.write("%s = %s\n" % (key, str(result[key])))
    return result

In [192]:
#import pdb

def eval_hdf5(size=100):
    '''
        Run this for evaluting parts of the evaluation set by specifing the 'size'
    '''
    args['output_dir'].mkdir(exist_ok=True)
    
    #This will support big dataset
    transformer = PatentTransformer(label_list, args['max_seq_length'], tokenizer)

    eval_data = HDF5Dataset(args['hdf5_dataset_file'], args['hdf5_eval_dataset_name'], transformer=transformer, \
                            size=size)
    
    eval_dataloader = DataLoader(eval_data, shuffle=False, batch_size=1, num_workers=0)
  
    
    all_logits = None
    all_labels = None
    
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    for input_ids, input_mask, segment_ids, label_ids in tqdm(eval_dataloader,total=size):
        
        #pdb.set_trace()
        
        
        input_ids = input_ids.to(device)
        input_mask = input_mask.to(device)
        segment_ids = segment_ids.to(device)
        label_ids = label_ids.to(device)

        with torch.no_grad():
            tmp_eval_loss = model(input_ids, segment_ids, input_mask, label_ids)
            logits = model(input_ids, segment_ids, input_mask)

#         logits = logits.detach().cpu().numpy()
#         label_ids = label_ids.to('cpu').numpy()
#         tmp_eval_accuracy = accuracy(logits, label_ids)
        tmp_eval_accuracy = accuracy_thresh(logits, label_ids)
        if all_logits is None:
            all_logits = logits.detach().cpu().numpy()
        else:
            all_logits = np.concatenate((all_logits, logits.detach().cpu().numpy()), axis=0)
            
        if all_labels is None:
            all_labels = label_ids.detach().cpu().numpy()
        else:    
            all_labels = np.concatenate((all_labels, label_ids.detach().cpu().numpy()), axis=0)
        

        eval_loss += tmp_eval_loss.mean().item()
        eval_accuracy += tmp_eval_accuracy

        nb_eval_examples += input_ids.size(0)
        nb_eval_steps += 1

    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_examples
    
    
    
#     ROC-AUC calcualation
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    
    for i in range(num_labels):
        fpr[i], tpr[i], _ = roc_curve(all_labels[:, i], all_logits[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
        
    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(all_labels.ravel(), all_logits.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

    result = {'eval_loss': eval_loss,
              'eval_accuracy': eval_accuracy,
#               'loss': tr_loss/nb_tr_steps,
              'roc_auc': roc_auc  }

    output_eval_file = os.path.join(args['output_dir'], "eval_results.txt")
    with open(output_eval_file, "w") as writer:
        logger.info("***** Eval results *****")
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))
            writer.write("%s = %s\n" % (key, str(result[key])))
    return result

### Train Model

In [ ]:
def fit(num_epocs=args['num_train_epochs']):
    global_step = 0
    model.train()
    for i_ in tqdm(range(int(num_epocs)), desc="Epoch"):

        tr_loss = 0
        nb_tr_examples, nb_tr_steps = 0, 0
        for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):

            batch = tuple(t.to(device) for t in batch)
            input_ids, input_mask, segment_ids, label_ids = batch
            #logger.debug(input_ids.shape, input_mask.shape, segment_ids.shape, label_ids.shape)
            loss = model(input_ids, segment_ids, input_mask, label_ids)
            if n_gpu > 1:
                loss = loss.mean() # mean() to average on multi-gpu.
            if args['gradient_accumulation_steps'] > 1:
                loss = loss / args['gradient_accumulation_steps']

            if args['fp16']:
                optimizer.backward(loss)
            else:
                loss.backward()

            tr_loss += loss.item()
            nb_tr_examples += input_ids.size(0)
            nb_tr_steps += 1
            if (step + 1) % args['gradient_accumulation_steps'] == 0:
    #             scheduler.batch_step()
                # modify learning rate with special warm up BERT uses
                lr_this_step = args['learning_rate'] * warmup_linear(global_step/t_total, args['warmup_proportion'])
                for param_group in optimizer.param_groups:
                    param_group['lr'] = lr_this_step
                optimizer.step()
                optimizer.zero_grad()
                global_step += 1

        logger.info('Loss after epoc {}'.format(tr_loss / nb_tr_steps))
        logger.info('Eval after epoc {}'.format(i_+1))
        eval()

In [ ]:
# Freeze BERT layers for 1 epoch
# model.module.freeze_bert_encoder()

In [ ]:
# fit(1)

In [ ]:
model.unfreeze_bert_encoder()

In [ ]:
model

BertForMultiLabelSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
 

In [ ]:
fit()

Iteration:  48%|████▊     | 76/157 [3:38:56<3:50:30, 170.74s/it]

In [ ]:
# Save a trained model
model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self
output_model_file = os.path.join(PYTORCH_PRETRAINED_BERT_CACHE, "finetuned_pytorch_bert_cpc_model.bin")
torch.save(model_to_save.state_dict(), output_model_file)

# Load a trained model that you have fine-tuned
model_state_dict = torch.load(output_model_file)
model = BertForMultiLabelSequenceClassification.from_pretrained(args['bert_model'], num_labels = num_labels, state_dict=model_state_dict)
model.to(device)

### Model Evaluation

In [ ]:
#Evaluate entire evalution set
eval()

In [114]:
#Evaluate the evaluation set with a size
eval_hdf5(size=10)



  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:02<00:22,  2.48s/it]

 20%|██        | 2/10 [00:07<00:25,  3.14s/it]

 30%|███       | 3/10 [00:09<00:20,  2.94s/it]

 40%|████      | 4/10 [00:12<00:16,  2.80s/it]

 50%|█████     | 5/10 [00:14<00:13,  2.71s/it]

 60%|██████    | 6/10 [00:17<00:10,  2.64s/it]

 70%|███████   | 7/10 [00:19<00:07,  2.59s/it]

 80%|████████  | 8/10 [00:22<00:05,  2.55s/it]

 90%|█████████ | 9/10 [00:24<00:02,  2.56s/it]

100%|██████████| 10/10 [00:27<00:00,  2.53s/it]



In [73]:
def predict(model, test_filename):
    
    
    #Read small dataset
    dataframe = pd.read_json(test_filename, orient='records', lines=True)
    
    #This will support big dataset
    transformer = PatentTransformer(label_list, args['max_seq_length'], tokenizer)

  
    test_data = PatentDataset(dataframe,transformer=transformer, size=100)
    test_dataloader = DataLoader(test_data, shuffle=False, batch_size=1)

  
    
    all_logits = None
    
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    for step, batch in enumerate(tqdm(test_dataloader, desc="Prediction Iteration")):
        input_ids, input_mask, segment_ids, label_ids = batch
        input_ids = input_ids.to(device)
        input_mask = input_mask.to(device)
        segment_ids = segment_ids.to(device)

        with torch.no_grad():
            logits = model(input_ids, segment_ids, input_mask)
            logits = logits.sigmoid()

        if all_logits is None:
            all_logits = logits.detach().cpu().numpy()
        else:
            all_logits = np.concatenate((all_logits, logits.detach().cpu().numpy()), axis=0)
            
        nb_eval_examples += input_ids.size(0)
        nb_eval_steps += 1

    return pd.merge(dataframe, pd.DataFrame(all_logits, columns=label_list), left_index=True, right_index=True)

In [65]:
DATA_PATH

PosixPath('/home/bimhud/bert-cpc')

In [143]:
test_filename = os.path.join(DATA_PATH,'train_top_10.json')

In [144]:
result = predict(model, test_filename)



Prediction Iteration:   0%|          | 0/10 [00:00<?, ?it/s]

Prediction Iteration:  10%|█         | 1/10 [00:01<00:11,  1.32s/it]

Prediction Iteration:  20%|██        | 2/10 [00:02<00:10,  1.31s/it]

Prediction Iteration:  30%|███       | 3/10 [00:03<00:09,  1.31s/it]

Prediction Iteration:  40%|████      | 4/10 [00:05<00:07,  1.30s/it]

Prediction Iteration:  50%|█████     | 5/10 [00:06<00:06,  1.30s/it]

Prediction Iteration:  60%|██████    | 6/10 [00:07<00:05,  1.29s/it]

Prediction Iteration:  70%|███████   | 7/10 [00:09<00:03,  1.29s/it]

Prediction Iteration:  80%|████████  | 8/10 [00:10<00:02,  1.30s/it]

Prediction Iteration:  90%|█████████ | 9/10 [00:11<00:01,  1.30s/it]

Prediction Iteration: 100%|██████████| 10/10 [00:12<00:00,  1.30s/it]



# Get top N performance

In [149]:
topn=3
columns = result.columns[7:]
values = result.loc[:,columns]
indexes = np.argsort(1-result.loc[:,columns],axis=1)


In [150]:
top_n_result = []
for row_id in range(len(result)):
    idx = indexes.loc[row_id,:][0:topn]
    top_n_predicted = dict(zip(columns[idx], values.loc[0,columns[idx]]))
    top_n_result.append(top_n_predicted)
result.insert(0,'predict',top_n_result)

In [151]:
result.head()

,predict,abstractText,claimText,cpcList,descriptionText,pubId,titleText,A01B1/00,A01B1/02,A01B1/022,...,H05K9/0088,H05K9/009,H05K9/0092,H05K9/0094,H05K9/0096,H05K9/0098,H05K999/00,H05K999/99,Y02E10/00,Y02E50/16
0,"{'A61C13/0835': 0.0741344541311264, 'F01N3/087...",A microfluidic device is provided for inducing...,[1. A method for separating a constituent elem...,"[B01D9/04, B01L3/502769, B01D9/004, B01D17/00,...",[This application is a continuation under 35 U...,US9468867B2,Microfluidic device for inducing separations b...,0.049855,0.058862,0.048949,...,0.040075,0.047280,0.043040,0.031185,0.039559,0.035959,0.033472,0.039938,0.049472,0.039632
1,"{'A61C13/0835': 0.0741344541311264, 'F01N3/087...",Methods and apparatuses are provided for separ...,[1. An apparatus comprising at least one vesse...,"[F26B5/00, B01D9/0081, B01D33/0338, C02F11/12,...",[This application claims priority to U.S. Prov...,US9140493B2,Apparatus and process for concentrating a sepa...,0.049873,0.058882,0.049049,...,0.040158,0.047211,0.043005,0.031176,0.039560,0.035982,0.033413,0.039935,0.049481,0.039640
2,"{'A61C13/0835': 0.0741344541311264, 'F01N3/087...",Disclosed is method for separating solute from...,[1. A method for separating solute from an aqu...,"[C01D3/04, B01D9/04, B01D9/0013]",[This application claims the benefit of U.S. P...,US8454926B2,Method and system for separation of solute fro...,0.049879,0.058877,0.049033,...,0.040144,0.047221,0.043011,0.031176,0.039560,0.035976,0.033425,0.039942,0.049479,0.039636
3,"{'A61C13/0835': 0.0741344541311264, 'F01N3/087...",The present invention provides a process and a...,[1. A process for concentrating dilute solutio...,"[B01D9/004, B01D9/004, B01D9/0045, B01D9/04, B...",[The present invention relates to a process an...,US8245521B2,Process and apparatus for concentrating dilute...,0.049868,0.058884,0.049035,...,0.040147,0.047223,0.043009,0.031179,0.039563,0.035983,0.033415,0.039932,0.049478,0.039640
4,"{'A61C13/0835': 0.0741344541311264, 'F01N3/087...",A continuous in-line process of making fragran...,[1. A continuous in-line process for making a ...,"[A61K8/922, A61K8/925, B01D9/0004, B01D9/0059,...",[The present invention relates to methods of m...,US20160376520A1,Continuous In-Line Process for Making Fragranc...,0.049818,0.058925,0.049059,...,0.040168,0.047226,0.042993,0.031203,0.039584,0.036015,0.033355,0.039888,0.049477,0.039661


In [86]:
result.loc[0,'B01D9/004']

0.044620257

In [50]:
result.loc[0,'C08L33/10']

0.045590457

In [53]:
result.loc[0,'A61C13/0835']

0.07409361

Finished